In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdmolfiles
from joblib import Parallel, delayed

The reaction SMILES database is a .csv file containing SMILES in a column with header 'ReactionSmiles'

In [ ]:
class GenerateTemplates():
    def __init__(self,path,key ='ReactionSmiles'):
        uspto=pd.read_csv (path,low_memory=False)
        self.reactsmiles=uspto[key].tolist()
        
        
    def generatereactsmarts(self,reactsmiles):
        ''' Input : SMILES of reaction (string) ------------> Output :Template of reaction (string) if success
                                                                      False  if Failure '''
        reactsmarts=[]
        mapnum=[]
        ref=[]
        try:
            # Check Parsability of Reaction SMILES
            rxn=AllChem.ReactionFromSmarts(reactsmiles)
            rxn.Initialize()
        except:
            return False

        # Obtain reactant templates
        reactants=[rxn.GetReactantTemplate(m) for m in range(rxn.GetNumReactantTemplates())]
        # Obtain product templates
        products=[rxn.GetProductTemplate(m) for m in range(rxn.GetNumProductTemplates())]
        try:
            # Identify reacting atoms
            reactatoms=rxn.GetReactingAtoms(mappedAtomsOnly=True) # sequence of sequences with the atoms that change in the reaction
        except:
            return False
        newmap=1

        # Renumber atom mapping  numbers of reacting atoms to start from 1
        for i in range(len(reactants)):
            atoms=reactants[i].GetAtoms()
            id=[]
            for j in reactatoms[i]:
                id.append(j)
                mapnum.append(atoms[j].GetAtomMapNum()) # Get AAM number of the reactant atom
                atoms[j].SetAtomMapNum(newmap) # Change AAM number
                ref.append(newmap)
                newmap+=1
                neighbors=atoms[j].GetNeighbors() #Obtain neighbors of reacting atoms
                for n in neighbors:
                    if n.GetIdx() not in reactatoms[i]: #Check if the neighbor of a reacting atom is not a reacting atom as well
                        if n.GetAtomMapNum() !=0:
                            id.append(n.GetIdx())
                            mapnum.append(n.GetAtomMapNum())
                            n.SetAtomMapNum(newmap)
                            ref.append(newmap)
                            newmap+=1
            try:
                if not id:
                    continue
                reactstring=rdmolfiles.MolFragmentToSmarts(reactants[i],atomsToUse=id,isomericSmarts=False) #Create SMARTS of template for 1 reactant
            except:
                return False
            reactstring='('+reactstring+')'
            if i == 0 or not reactsmarts:
                reactsmarts=reactstring
                continue
            reactsmarts='{}.{}'.format(reactsmarts,reactstring) # Combine strings of each reactant
        if not reactsmarts :
            return False
        reactsmarts=reactsmarts+'>>'


        for i in range(len(products)):
            atoms=products[i].GetAtoms()
            string=Chem.MolToSmiles(products[i])
            p=Chem.MolFromSmiles(string)

            index=[a.GetIdx() for a in atoms if a.GetAtomMapNum() in mapnum] # Get ids of atoms that have been identified as reaction centre or neighbor
            for ind in index:
                dummy= mapnum.index(atoms[ind].GetAtomMapNum())
                atoms[ind].SetAtomMapNum(ref[dummy]) # Set corresponding Atom Map num for product atom from reactant atom
            prodid=index

            try:
                if not prodid:
                    continue
                prodstring = rdmolfiles.MolFragmentToSmarts(products[i], atomsToUse=prodid,isomericSmarts=False) #Create SMARTS of template for 1 product
            except:
                return False
            prodstring='('+prodstring+')'
            if i ==0:
                reactsmarts=reactsmarts+prodstring
            continue
            reactsmarts='{}.{}'.format(reactsmarts,prodstring) # combine SMARTS of each product
        return(reactsmarts)
    
    def runtemplategen(self,path):
        reactsmarts=Parallel(n_jobs=-1,verbose=1)(delayed(self.generatereactsmarts)(r) for r in self.reactsmiles)
        dict={'Reaction Smarts':reactsmarts,'ReactionSmiles':reactsmiles}
        df=pd.DataFrame(dict)
        df.to_csv(path,index=None)
        
        
